In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from google.cloud import bigquery
from witwidget.notebook.visualization import WitWidget, WitConfigBuilder

In [2]:
query="""
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks
FROM
  publicdata.samples.natality
WHERE year > 2000
LIMIT 10000
"""
df = bigquery.Client().query(query).to_dataframe()
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks
0,7.063611,True,32,1,37.0
1,4.687028,True,30,3,33.0
2,7.561856,True,20,1,39.0
3,7.561856,True,31,1,37.0
4,7.312733,True,32,1,40.0


In [3]:
df.describe()


,weight_pounds,mother_age,plurality,gestation_weeks
count,9997.000000,10000.000000,10000.000000,9886.000000
mean,7.303938,27.226500,1.033500,38.700890
std,1.313289,6.194646,0.192825,2.538734
min,0.595248,13.000000,1.000000,18.000000
25%,6.624891,22.000000,1.000000,38.000000
50%,7.374463,27.000000,1.000000,39.000000
75%,8.126239,32.000000,1.000000,40.000000
max,12.625874,52.000000,4.000000,47.000000


In [4]:
df['is_male'].value_counts()

True     5096
False    4904
Name: is_male, dtype: int64

In [5]:
df = df.dropna()
df = shuffle(df, random_state=2)

In [6]:
labels = df['weight_pounds']
data = df.drop(columns=['weight_pounds'])
data['is_male'] = data['is_male'].astype(int)

In [7]:
data.head()

,is_male,mother_age,plurality,gestation_weeks
40,0,18,1,39.0
5976,0,22,1,47.0
6902,0,29,1,38.0
4838,0,32,1,37.0
8335,1,29,1,39.0


In [8]:
x,y = data,labels
x_train,x_test,y_train,y_test = train_test_split(x,y)

In [9]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(x_train.iloc[0]),)),
    Dense(32, activation='relu'),
    Dense(1)]
)

2021-10-14 13:49:14.362632: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-10-14 13:49:14.364263: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x56006b8bfcb0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-10-14 13:49:14.364300: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-10-14 13:49:14.367509: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [10]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mae', 'mse'])

In [11]:
model.fit(x_train, y_train, epochs=10, validation_split=0.1)

Train on 6671 samples, validate on 742 samples
Epoch 1/10
6671/6671 [==============================] - 2s 337us/sample - loss: 1.6941 - mae: 0.9890 - mse: 1.6941 - val_loss: 1.5937 - val_mae: 0.9799 - val_mse: 1.5937
Epoch 2/10
6671/6671 [==============================] - 1s 116us/sample - loss: 1.4233 - mae: 0.9353 - mse: 1.4233 - val_loss: 1.3808 - val_mae: 0.9342 - val_mse: 1.3808
Epoch 3/10
6671/6671 [==============================] - 1s 148us/sample - loss: 1.4172 - mae: 0.9318 - mse: 1.4172 - val_loss: 1.3330 - val_mae: 0.8979 - val_mse: 1.3330
Epoch 4/10
6671/6671 [==============================] - 1s 205us/sample - loss: 1.3889 - mae: 0.9205 - mse: 1.3889 - val_loss: 1.2489 - val_mae: 0.8703 - val_mse: 1.2489
Epoch 5/10
6671/6671 [==============================] - 1s 179us/sample - loss: 1.3665 - mae: 0.9148 - mse: 1.3665 - val_loss: 1.4549 - val_mae: 0.9392 - val_mse: 1.4549
Epoch 6/10
6671/6671 [==============================] - 1s 151us/sample - loss: 1.3290 - mae: 0.9036 - 

In [12]:
num_examples = 10
predictions = model.predict(x_test[:num_examples])

In [13]:
for i in range(num_examples):
    print('Predicted val: ', predictions[i][0])
    print('Actual val: ',y_test.iloc[i])
    print()

Predicted val:  6.406772
Actual val:  5.43659938092

Predicted val:  5.7045217
Actual val:  5.5005334369

Predicted val:  6.8090863
Actual val:  8.2011961464

Predicted val:  6.285322
Actual val:  7.25100379718

Predicted val:  6.4895077
Actual val:  6.84976248034

Predicted val:  6.7692904
Actual val:  8.046872562999999

Predicted val:  6.216803
Actual val:  6.93794738514

Predicted val:  6.2682524
Actual val:  6.1949895622

Predicted val:  6.357265
Actual val:  7.936641432

Predicted val:  6.9858203
Actual val:  5.74965579296



In [14]:
wit_data = pd.concat([x_test, y_test], axis=1)

In [15]:
def custom_predict(examples_to_infer):
    preds = model.predict(examples_to_infer)
    return preds

In [16]:
config_builder = (WitConfigBuilder(wit_data[:500].values.tolist(), data.columns.tolist() + ['weight_pounds'])
  .set_custom_predict_fn(custom_predict)
  .set_target_feature('weight_pounds')
  .set_model_type('regression'))
WitWidget(config_builder, height=800)

WitWidget(config={'model_type': 'regression', 'label_vocab': [], 'feature_names': ['is_male', 'mother_age', 'p…